In [1]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm_notebook # check the progressbar in the python. 
import glob # check the file name in fold. 

In [2]:
import pyspark

import findspark
findspark.init()
findspark.find()

'C:\\spark-2.4.4-bin-hadoop2.7\\spark-2.4.4-bin-hadoop2.7'

In [3]:
# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'

In [4]:
read_file_lst = glob.glob(path + '*')
read_file_lst[0:2]

['C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final\\log_20100501.csv',
 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final\\log_20100502.csv']

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [7]:
from pyspark.sql import SQLContext
from pyspark import SparkFiles
from pyspark.sql.functions import lit

sqlContext = SQLContext(sc)
for i in tqdm_notebook(read_file_lst):
    # sqlContext.read.csv() : path를 C:Users/..의 형식으로 맞춰줘야함. (초기값이 spark설치된 곳으로 지정되어있음)
    df_temp = sqlContext.read.csv(SparkFiles.get(i), header=True, inferSchema= True) # read.csv 
    
    # transformation으로 추가하려면, df = df.withColumn("age_square", col("age")**2)
    df_temp = df_temp.withColumn("date", lit(i[-8:-4])) # column 추가, lit명령어가 value를 추가하는 방법. 
    
    if i == read_file_lst[0]:
        df_total = df_temp
    else:
        df_total = unionAll([df_total, df_temp]) # row-wise 결합
        
    del df_temp 
    gc.collect()

In [8]:
import pyspark.sql.functions as f
# userItem=df.groupby('userId').agg(f.expr('count(distinct item)').alias('n_item'))
# df_total.groupBy("log_id").agg(f.expr('count(distinct item)').alias('log_id'))
# df_total.select("actor_account").distinct().count()

### Feature Engineering

Player action features
### Ratio는 무슨의미인지 잘 몰라서 아직 코드를 안짰습니다. 
- Sit (count / ratio / per_day_count)
- Exp_get
- item_get
- money_get
- abyss_get
- Exp_repair (count / per_day_count)
- Use_portal
- Killed_bypc
- Killed_bynpc
- Teleport
- Reborn

In [ ]:
import pyspark.sql.functions as func # pyspark의 유용한 기능을 사용하는 패키지 : countDistinct 사용가능
# 원하는 부분만 출력할때는 df.filter()를 사용. 

# 아이온데이터에 actor는 다르지만 actor_account가 같은 경우는 있는 것 같음. 
# 하지만 제출형태가 actor_account를 기준으로 해가지고 groupby를 아래의 변수로 진행했음. 
df_total = df_total.withColumnRenamed("actor_account", "account")

In [ ]:
mvv_list = df_total.select('account').distinct().collect()
mvv_array = [int(row.account) for row in mvv_list.collect()]
total_label = pd.DataFrame()
total_label['account'] = mvv_array

In [ ]:
total_label

In [ ]:
def action_features(train, test, df, name, count, ratio, per_day_count, log_id, etc_num):
    df_agg = df.filter(df['log_id'] == log_id).select('account', 'date').toPandas()
    df_agg = df_agg.groupby(['account', 'date'])['account'].agg({'count'}).reset_index()
    df_agg = df_agg.groupby(['account'])['count'].agg({'mean', 'sum'}).reset_index()
    df_agg = df_agg.rename(columns={'mean':'{}_per_day_count'.format(name), 'sum':'{}_sum_count'.format(name), 'std':'{}_std_per_day_count'.format(name)})
    train = train.merge(df_agg, on='account' , how='left')
    test = test.merge(df_agg, on='account' , how='left')
        
    df_agg = df.filter(df['log_id'] == log_id).select('account').toPandas()
    df_agg = df_agg.groupby(['account'])['account'].agg({'count'}).reset_index().rename(columns={'count':"{}_count".format(name)})
    train = train.merge(df_agg, on='account' , how='left')
    test = test.merge(df_agg, on='account' , how='left')

    print("Processing end...")
    return train, test

In [ ]:
# sit information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'sit', log_id = 118 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# Exp_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Exp_get', log_id = 143 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# item_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'item_get', log_id = 225 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# money_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'money_get', log_id = 187 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# abyss_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'abyss_get', log_id = 156 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features/'
train_label.to_csv(outputpath + "train_action1.csv", index=False)
test_label.to_csv(outputpath + "test_action1.csv", index=False)